In [114]:
import pandas as pd
import numpy as np
from sklearn.preprocessing import MinMaxScaler
from sklearn.metrics.pairwise import cosine_similarity, euclidean_distances, manhattan_distances

In [115]:
# df = pd.read_csv('1980.csv')
df = pd.read_csv('data/DE/data.csv')

In [116]:
df.head()

,name,artists,album,release_date,release_date_precision,chart_power,spotify_id,uri,popularity,danceability,...,speechiness,acousticness,instrumentalness,liveness,valence,tempo,duration_ms,time_signature,isrc,genres
0,That's No Way To Get Along,Robert Wilkins,The Original Rolling Stone,1980-01-01,day,NaN,5JKdsNzhJGpoaKBFZpOKQ3,spotify:track:5JKdsNzhJGpoaKBFZpOKQ3,22,0.427,...,0.0407,0.980,0.00000,0.1920,0.381,97.078,173827,4,USA370640304,acoustic
1,I Got A Problem,Albert Collins,Frostbite,1980-02-05,day,NaN,6e2PhVePvqe7w4VNBTslef,spotify:track:6e2PhVePvqe7w4VNBTslef,15,0.760,...,0.0573,0.328,0.03570,0.0447,0.849,102.183,274707,4,USARL8071903,acoustic
2,The Highway Is Like A Woman,Albert Collins,Frostbite,1980-02-05,day,NaN,4Qzcy6jU0yeck7IuCqGWSi,spotify:track:4Qzcy6jU0yeck7IuCqGWSi,16,0.751,...,0.0361,0.419,0.11000,0.1050,0.771,98.545,302600,4,USARL8071904,acoustic
3,Alabama Blues,Robert Wilkins,The Original Rolling Stone,1980-01-01,day,NaN,42zUiYaKltE7jFWb57fXAW,spotify:track:42zUiYaKltE7jFWb57fXAW,11,0.490,...,0.0716,0.972,0.00000,0.1590,0.905,205.718,157933,3,USA370640301,acoustic
4,Get Away Blues,Robert Wilkins,The Original Rolling Stone,1980-01-01,day,NaN,0hCrfp9WImVO6KWI30O1sM,spotify:track:0hCrfp9WImVO6KWI30O1sM,4,0.629,...,0.0431,0.968,0.00238,0.0941,0.322,119.752,213107,4,USA370640300,acoustic


In [117]:
df['genres'].unique()

array(['acoustic', 'afrobeat', 'alt-rock', 'alternative', 'ambient',
       'anime', 'bluegrass', 'blues', 'brazil', 'british', 'cantopop',
       'chicago-house', 'children', 'chill', 'classical', 'club',
       'comedy', 'country', 'dance', 'dancehall', 'disco', 'dub',
       'dubstep', 'edm', 'electro', 'electronic', 'folk', 'forro',
       'french', 'funk', 'garage', 'german', 'gospel', 'goth', 'groove',
       'grunge', 'guitar', 'hard-rock', 'hardcore', 'hardstyle',
       'heavy-metal', 'hip-hop', 'honky-tonk', 'house', 'indian', 'indie',
       'indie-pop', 'industrial', 'j-dance', 'j-idol', 'j-pop', 'j-rock',
       'jazz', 'k-pop', 'latin', 'latino', 'malay', 'mandopop', 'metal',
       'minimal-techno', 'mpb', 'new-age', 'opera', 'pagode', 'party',
       'piano', 'pop', 'pop-film', 'power-pop', 'progressive-house',
       'psych-rock', 'punk', 'punk-rock', 'r-n-b', 'reggae', 'reggaeton',
       'rock', 'rock-n-roll', 'rockabilly', 'salsa', 'samba', 'sertanejo',
       'sing

Check for duplicate values

In [118]:
df.shape

(349308, 24)

In [119]:
df.drop_duplicates(inplace=True)

In [120]:
df.shape

(83286, 24)

There were a lot of duplicate values..

Check for null values

In [121]:
df.isna().any()

name                      False
artists                   False
album                     False
release_date              False
release_date_precision    False
chart_power                True
spotify_id                False
uri                       False
popularity                False
danceability              False
energy                    False
key                       False
loudness                  False
mode                      False
speechiness               False
acousticness              False
instrumentalness          False
liveness                  False
valence                   False
tempo                     False
duration_ms               False
time_signature            False
isrc                      False
genres                    False
dtype: bool

There exist Null values for the Chart Power and for the isrc. The null values for the Chart Power are valid, because not every song was in the Charts. Lets have a look at the null values for isrc:

In [122]:
df[df['isrc'].isna()]

,name,artists,album,release_date,release_date_precision,chart_power,spotify_id,uri,popularity,danceability,...,speechiness,acousticness,instrumentalness,liveness,valence,tempo,duration_ms,time_signature,isrc,genres


Check for duplicate isrc numbers

In [123]:
df.columns

Index(['name', 'artists', 'album', 'release_date', 'release_date_precision',
       'chart_power', 'spotify_id', 'uri', 'popularity', 'danceability',
       'energy', 'key', 'loudness', 'mode', 'speechiness', 'acousticness',
       'instrumentalness', 'liveness', 'valence', 'tempo', 'duration_ms',
       'time_signature', 'isrc', 'genres'],
      dtype='object')

First group the songs by the spotify id. This brings together all instances with different genres but same spotify id.

In [391]:
by_spotify_id = df.copy()
by_spotify_id = by_spotify_id.groupby('spotify_id').agg({
    'genres': list,
    'name': list,
    'artists': list,
    'album': list,
    'release_date': list,
    'release_date_precision': list,
    'uri': list,
    'isrc':list,
    'chart_power': 'mean', 
    'popularity': 'mean', 
    'danceability': 'mean', 
    'energy': 'mean',
    'key': 'mean', 
    'loudness': 'mean', 
    'mode': 'mean', 
    'speechiness': 'mean', 
    'acousticness': 'mean', 
    'instrumentalness': 'mean', 
    'liveness': 'mean', 
    'valence': 'mean',
    'tempo': 'mean', 
    'duration_ms': 'mean', 
    'time_signature': 'mean'
})

In [392]:
by_spotify_id.head()

,genres,name,artists,album,release_date,release_date_precision,uri,isrc,chart_power,popularity,...,loudness,mode,speechiness,acousticness,instrumentalness,liveness,valence,tempo,duration_ms,time_signature
spotify_id,,,,,,,,,,,,,,,,,,,,,
000P83HDtOHcNVFZy7Q2Yu,[salsa],[El Licor de Tu Boquita],[El Gran Combo De Puerto Rico],[Unity],[1980],[year],[spotify:track:000P83HDtOHcNVFZy7Q2Yu],[USMRE0801806],NaN,21.0,...,-8.377,0.0,0.0736,0.438,0.000042,0.126,0.696,88.940,242493.0,4.0
000paUq6xoYcvCz2REJdu1,"[pop, pop-film]","[Hun Tey Kharkan Gay Alarum, Hun Tey Kharkan G...","[Noor Jehan, Noor Jehan]","[Noor Jehan New Punjabi Songs, Noor Jehan New ...","[1980-12-01, 1980-12-01]","[day, day]","[spotify:track:000paUq6xoYcvCz2REJdu1, spotify...","[USA561127553, USA561127553]",NaN,0.0,...,-9.944,1.0,0.0868,0.741,0.037700,0.222,0.606,96.169,372167.0,4.0
0017A9myIXhXYB4gBsPRsA,[pop],[Untuk Sebuah Nama],[Ratih Purwasih],[Untuk Sebuah Nama],[1980],[year],[spotify:track:0017A9myIXhXYB4gBsPRsA],[UKFT41826340],NaN,0.0,...,-3.549,1.0,0.0289,0.084,0.000004,0.117,0.360,81.433,249504.0,4.0
001reycpvPkUjmpxTJv6eV,"[classical, german]","[Oboe Concerto in F Minor, TWV 51:f1: I. Alleg...","[Georg Philipp Telemann,Burkhard Glaetzner,Har...",[Oboe Concertos (Baroque) - Antonio Vivaldi: G...,"[1980, 1980]","[year, year]","[spotify:track:001reycpvPkUjmpxTJv6eV, spotify...","[DES380744015, DES380744015]",NaN,0.0,...,-14.759,0.0,0.0348,0.831,0.916000,0.129,0.540,112.185,215733.0,4.0
002H0J6N0wKjjRtFQhKDzZ,"[dance, turkish]","[Arabesk Oyun Havası, Arabesk Oyun Havası]","[Esin Engin Orkestrası, Esin Engin Orkestrası]","[Arabesk'80, Arabesk'80]","[1980, 1980]","[year, year]","[spotify:track:002H0J6N0wKjjRtFQhKDzZ, spotify...","[BGA472127386, BGA472127386]",NaN,0.0,...,-12.444,0.0,0.0365,0.528,0.323000,0.343,0.752,134.274,345943.0,4.0


Then reduce all categorical features such that there are no duplicated values in an instance for one feature.

In [366]:
def reduce_list(elements, string_return = True):
    '''
    Removes duplicate elements in a list

    Parameter
    ---------
    elements: list
        List that should be reduced

    string_return: boolean; default=True
        Whether a list with just one element should be returned as string or list

    Return
    ------
    unique_elements: list or str
    
    '''
    unique_elements = []
    for element in elements:
        if element not in unique_elements:
            unique_elements.append(element)
    if (len(unique_elements) == 1 and string_return):
        return unique_elements[0]
    return unique_elements

In [367]:
by_spotify_id['isrc'] = by_spotify_id['isrc'].agg(reduce_list)
by_spotify_id['name'] = by_spotify_id['name'].agg(reduce_list)
by_spotify_id['artists'] = by_spotify_id['artists'].agg(reduce_list)
by_spotify_id['album'] = by_spotify_id['album'].agg(reduce_list)
by_spotify_id['release_date'] = by_spotify_id['release_date'].agg(reduce_list)
by_spotify_id['release_date_precision'] = by_spotify_id['release_date_precision'].agg(reduce_list)
by_spotify_id['uri'] = by_spotify_id['uri'].agg(reduce_list)
by_spotify_id['genres'] = by_spotify_id['genres'].agg(reduce_list)

In [368]:
by_spotify_id.head()

,genres,name,artists,album,release_date,release_date_precision,uri,isrc,chart_power,popularity,...,loudness,mode,speechiness,acousticness,instrumentalness,liveness,valence,tempo,duration_ms,time_signature
spotify_id,,,,,,,,,,,,,,,,,,,,,
000P83HDtOHcNVFZy7Q2Yu,salsa,El Licor de Tu Boquita,El Gran Combo De Puerto Rico,Unity,1980,year,spotify:track:000P83HDtOHcNVFZy7Q2Yu,USMRE0801806,NaN,21.0,...,-8.377,0.0,0.0736,0.438,0.000042,0.126,0.696,88.940,242493.0,4.0
000paUq6xoYcvCz2REJdu1,"[pop, pop-film]",Hun Tey Kharkan Gay Alarum,Noor Jehan,Noor Jehan New Punjabi Songs,1980-12-01,day,spotify:track:000paUq6xoYcvCz2REJdu1,USA561127553,NaN,0.0,...,-9.944,1.0,0.0868,0.741,0.037700,0.222,0.606,96.169,372167.0,4.0
0017A9myIXhXYB4gBsPRsA,pop,Untuk Sebuah Nama,Ratih Purwasih,Untuk Sebuah Nama,1980,year,spotify:track:0017A9myIXhXYB4gBsPRsA,UKFT41826340,NaN,0.0,...,-3.549,1.0,0.0289,0.084,0.000004,0.117,0.360,81.433,249504.0,4.0
001reycpvPkUjmpxTJv6eV,"[classical, german]","Oboe Concerto in F Minor, TWV 51:f1: I. Allegro","Georg Philipp Telemann,Burkhard Glaetzner,Hart...",Oboe Concertos (Baroque) - Antonio Vivaldi: Ge...,1980,year,spotify:track:001reycpvPkUjmpxTJv6eV,DES380744015,NaN,0.0,...,-14.759,0.0,0.0348,0.831,0.916000,0.129,0.540,112.185,215733.0,4.0
002H0J6N0wKjjRtFQhKDzZ,"[dance, turkish]",Arabesk Oyun Havası,Esin Engin Orkestrası,Arabesk'80,1980,year,spotify:track:002H0J6N0wKjjRtFQhKDzZ,BGA472127386,NaN,0.0,...,-12.444,0.0,0.0365,0.528,0.323000,0.343,0.752,134.274,345943.0,4.0


The next step is to group all the songs by the isrc number since this should be the unique identifier. 

In [378]:
by_isrc = by_spotify_id.copy()
by_isrc = by_isrc[['isrc', 'name']]
by_isrc['count'] = 0
by_isrc = by_isrc.groupby('isrc').agg({
    'name': list,
    'count': 'count'
})

by_isrc.sort_values('count', ascending=False)

,name,count
isrc,,
USSM18000294,"[Mallorca (Barcarola), Op. 202 [Arranged by Jo...",7
USSM18000291,"[Suite Española No. 1, Op. 47: No. 1, Granada ...",7
USSM18000295,"[Cantos de España, Op. 232: No. 4, Córdoba (Ar...",7
USSM18000293,"[Suite Española No. 1, Op. 47: No. 3, Sevilla ...",6
USSM18000292,"[Suite Española No. 1, Op. 47: No. 5, Asturias...",6
...,...,...
GB2DY1000060,[Baphomet - Demo],1
GB2DY1000061,[Sorcerers - Demo],1
GB2DY1000062,[Extermination Day - Demo],1


In [379]:
by_isrc['name'] = by_isrc['name'].agg(reduce_list, string_return=False)

In [380]:
special_cases = by_isrc[by_isrc['name'].apply(len) > 1]

In [383]:
special_cases['name'] = special_cases['name'].apply(reduce_list)

/var/folders/t4/w1slb7fj5kg0tc41jhrm3zx00000gn/T/ipykernel_2586/102079820.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  special_cases['name'] = special_cases['name'].apply(reduce_list)


In [384]:
special_cases

,name,count
isrc,,
BRGVR1900214,"[Comparação, Conversão do Padre João]",2
BRGVR1900215,"[O Bom Cristão, Eu Vou para o Céu]",2
BRGVR1900216,"[Eu Sou, Jesus de Nazaré]",2
BRGVR1900217,"[Escondido em Jesus, Vencedor]",2
BRGVR1900218,"[As Duas Ovelhas, Alma Transformada]",2
...,...,...
USSM10411835,"[Cello Concerto No. 2 in D Major, Hob. VIIb:2:...",5
USSM10411836,"[Cello Concerto No. 2 in D Major, Hob. VIIb:2:...",5
USSM17900406,"[Cello Concerto No. 1 in C Major, Hob. VIIb:1:...",4


In [385]:
indices = special_cases.index


In [389]:
by_isrc.drop(index=indices.values)

,name,count
isrc,,
ARA821922001,[Tierra Negra],1
ARA821922002,[El Incendio],1
ARA821922003,[Patotero Sentimental],1
ARA821922004,[A Media Luz],1
ARA821922005,[Portero Suba y Diga],1
...,...,...
usx9p2035276,[Bad News Travels Fast],1
usx9p2035277,[All Tied Up],1
usx9p2035278,[Givin' It All],1


Group the DataFrame by the spotify id. All catgeorical values of the features are stored as lists. All numerical features are stored as the mean number.

In [100]:
df_total = df.copy()

df_total = df_total.groupby('spotify_id').agg({
    'spotify_id': list,
    'genres': list,
    'name': list,
    'artists': list,
    'album': list,
    'release_date': list,
    'release_date_precision': list,
    'uri': list,
    'isrc':list,
    'chart_power': 'mean', 
    'popularity': 'mean', 
    'danceability': 'mean', 
    'energy': 'mean',
    'key': 'mean', 
    'loudness': 'mean', 
    'mode': 'mean', 
    'speechiness': 'mean', 
    'acousticness': 'mean', 
    'instrumentalness': 'mean', 
    'liveness': 'mean', 
    'valence': 'mean',
    'tempo': 'mean', 
    'duration_ms': 'mean', 
    'time_signature': 'mean'
})

In [106]:
display(df_total.head())
df_total.shape

,spotify_id,genres,name,artists,album,release_date,release_date_precision,uri,isrc,chart_power,...,loudness,mode,speechiness,acousticness,instrumentalness,liveness,valence,tempo,duration_ms,time_signature
spotify_id,,,,,,,,,,,,,,,,,,,,,
000P83HDtOHcNVFZy7Q2Yu,[000P83HDtOHcNVFZy7Q2Yu],[salsa],[El Licor de Tu Boquita],[El Gran Combo De Puerto Rico],[Unity],[1980],[year],[spotify:track:000P83HDtOHcNVFZy7Q2Yu],[USMRE0801806],NaN,...,-8.377,0.0,0.0736,0.438,0.000042,0.1260,0.696,88.940,242493.0,4.0
000blRZoW279EJlePCHsoi,"[000blRZoW279EJlePCHsoi, 000blRZoW279EJlePCHsoi]","[folk, pop]","[La hermosura de una flor, La hermosura de una...","[Jose Ali Nieves, Jose Ali Nieves]","[Estampas De Puerto Ayacucho, Estampas De Puer...","[1981-04-30, 1981-04-30]","[day, day]","[spotify:track:000blRZoW279EJlePCHsoi, spotify...","[COA6W2101386, COA6W2101386]",NaN,...,-8.698,1.0,0.0989,0.374,0.000000,0.0946,0.811,89.951,197042.0,3.0
000paUq6xoYcvCz2REJdu1,"[000paUq6xoYcvCz2REJdu1, 000paUq6xoYcvCz2REJdu1]","[pop, pop-film]","[Hun Tey Kharkan Gay Alarum, Hun Tey Kharkan G...","[Noor Jehan, Noor Jehan]","[Noor Jehan New Punjabi Songs, Noor Jehan New ...","[1980-12-01, 1980-12-01]","[day, day]","[spotify:track:000paUq6xoYcvCz2REJdu1, spotify...","[USA561127553, USA561127553]",NaN,...,-9.944,1.0,0.0868,0.741,0.037700,0.2220,0.606,96.169,372167.0,4.0
0017A9myIXhXYB4gBsPRsA,[0017A9myIXhXYB4gBsPRsA],[pop],[Untuk Sebuah Nama],[Ratih Purwasih],[Untuk Sebuah Nama],[1980],[year],[spotify:track:0017A9myIXhXYB4gBsPRsA],[UKFT41826340],NaN,...,-3.549,1.0,0.0289,0.084,0.000004,0.1170,0.360,81.433,249504.0,4.0
0017lMteokXc5Mzcr2BYFp,"[0017lMteokXc5Mzcr2BYFp, 0017lMteokXc5Mzcr2BYF...","[classical, french, opera]","[Werther / Act 1: ""Assez! Assez!"" - ""Noël! Jés...","[Jules Massenet,Robert Lloyd,Malcolm King,Paul...","[Massenet: Werther, Massenet: Werther, Massene...","[1981-01-01, 1981-01-01, 1981-01-01]","[day, day, day]","[spotify:track:0017lMteokXc5Mzcr2BYFp, spotify...","[NLA507812202, NLA507812202, NLA507812202]",NaN,...,-24.768,1.0,0.0489,0.979,0.094100,0.0879,0.159,95.434,453893.0,3.0


(77436, 24)

Create a DataFrame which contains only the relevant features for the recommender system

In [22]:
df_recommender = df.drop(columns=['artists', 'genres','album', 'release_date', 'release_date_precision', 'chart_power', 'uri', 'popularity', 'name', 'isrc'])

Drop duplicate values..

In [23]:
df_recommender.shape

(76414, 14)

In [24]:
df_recommender.drop_duplicates(inplace=True)

In [25]:
df_recommender.shape

(38456, 14)

There were a lot of duplicates..

In [26]:
df_recommender.head()

,spotify_id,danceability,energy,key,loudness,mode,speechiness,acousticness,instrumentalness,liveness,valence,tempo,duration_ms,time_signature
0,5xgPirJuHEYxM3pMFPYtXG,0.600,0.118,3,-18.452,1,0.0402,0.977,0.072900,0.1100,0.557,98.429,278787,4
1,6vonkFQZRt9WHHfO6iU1AL,0.887,0.344,1,-11.985,1,0.0481,0.832,0.028000,0.3370,0.620,129.867,171445,4
2,7pFvU6cr3CURlNc8S9SxAp,0.433,0.197,8,-13.150,1,0.0324,0.993,0.003270,0.1090,0.804,85.316,155840,4
3,6CR1NaaMBPQUed2i5QRW4l,0.442,0.167,2,-14.996,1,0.0358,0.859,0.001730,0.0876,0.243,72.212,189427,4
4,1td7cg3olnhudXRKSASjRZ,0.468,0.205,2,-13.606,0,0.0491,0.778,0.000237,0.0832,0.374,114.904,209107,3


Since all features except the spotify_id are numerical features and the spotify_id should be an unique identifier, try to group by the id to reduce the DataFrame even more.

In [27]:
df_recommender = df_recommender.groupby(['spotify_id']).mean()

In [28]:
display(df_recommender.head())
df_recommender.shape

,danceability,energy,key,loudness,mode,speechiness,acousticness,instrumentalness,liveness,valence,tempo,duration_ms,time_signature
spotify_id,,,,,,,,,,,,,
000P83HDtOHcNVFZy7Q2Yu,0.619,0.561,7.0,-8.377,0.0,0.0736,0.438,0.000042,0.126,0.696,88.940,242493.0,4.0
000paUq6xoYcvCz2REJdu1,0.492,0.558,2.0,-9.944,1.0,0.0868,0.741,0.037700,0.222,0.606,96.169,372167.0,4.0
0017A9myIXhXYB4gBsPRsA,0.220,0.668,0.0,-3.549,1.0,0.0289,0.084,0.000004,0.117,0.360,81.433,249504.0,4.0
001reycpvPkUjmpxTJv6eV,0.448,0.224,5.0,-14.759,0.0,0.0348,0.831,0.916000,0.129,0.540,112.185,215733.0,4.0
002H0J6N0wKjjRtFQhKDzZ,0.286,0.480,2.0,-12.444,0.0,0.0365,0.528,0.323000,0.343,0.752,134.274,345943.0,4.0


(38456, 13)

The shape stayed the same, so there were no duplicate ids.

Scale the data, so every feature has the same influence.

In [29]:
scaler = MinMaxScaler()
df_recommender_scaled = scaler.fit_transform(df_recommender)
df_recommender_scaled = pd.DataFrame(df_recommender_scaled, columns=df_recommender.columns, index = df_recommender.index)

In [30]:
df_recommender_scaled.head()

,danceability,energy,key,loudness,mode,speechiness,acousticness,instrumentalness,liveness,valence,tempo,duration_ms,time_signature
spotify_id,,,,,,,,,,,,,
000P83HDtOHcNVFZy7Q2Yu,0.638803,0.561519,0.636364,0.798337,0.0,0.076507,0.439758,0.000042,0.114847,0.696,0.387169,0.067168,0.8
000paUq6xoYcvCz2REJdu1,0.507740,0.558516,0.181818,0.764918,1.0,0.090229,0.743976,0.037851,0.212072,0.606,0.418638,0.104717,0.8
0017A9myIXhXYB4gBsPRsA,0.227038,0.668637,0.000000,0.901301,1.0,0.030042,0.084336,0.000004,0.105732,0.360,0.354490,0.069198,0.8
001reycpvPkUjmpxTJv6eV,0.462332,0.224149,0.454545,0.662231,0.0,0.036175,0.834337,0.919679,0.117885,0.540,0.488358,0.059419,0.8
002H0J6N0wKjjRtFQhKDzZ,0.295150,0.480430,0.181818,0.711602,0.0,0.037942,0.530120,0.324297,0.334616,0.752,0.584514,0.097124,0.8


In [31]:
df_recommender_scaled.reset_index(inplace=True)
df_recommender.reset_index(inplace=True)

Convert every genre to a feature. If a song is part of a genre it should contain the value 1 otherwise 0.

In [32]:
ct = pd.crosstab(df['spotify_id'], df['genres'])
ct.reset_index(inplace=True)

In [33]:
display(ct.head())
ct.shape

genres,spotify_id,acoustic,afrobeat,alt-rock,alternative,ambient,anime,bluegrass,blues,brazil,...,sleep,songwriter,soul,spanish,swedish,synth-pop,tango,techno,trance,turkish
0,000P83HDtOHcNVFZy7Q2Yu,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,000paUq6xoYcvCz2REJdu1,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,0017A9myIXhXYB4gBsPRsA,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,001reycpvPkUjmpxTJv6eV,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,002H0J6N0wKjjRtFQhKDzZ,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,1


(38456, 96)

Merge both DataFrames together to create the Recommender System.

In [34]:
ct_merged = ct.merge(df_recommender_scaled, on=['spotify_id'], how='left')

In [35]:
ct_merged.set_index(['spotify_id'], inplace=True)

In [36]:
display(ct_merged.head())
ct_merged.shape

,acoustic,afrobeat,alt-rock,alternative,ambient,anime,bluegrass,blues,brazil,british,...,loudness,mode,speechiness,acousticness,instrumentalness,liveness,valence,tempo,duration_ms,time_signature
spotify_id,,,,,,,,,,,,,,,,,,,,,
000P83HDtOHcNVFZy7Q2Yu,0,0,0,0,0,0,0,0,0,0,...,0.798337,0.0,0.076507,0.439758,0.000042,0.114847,0.696,0.387169,0.067168,0.8
000paUq6xoYcvCz2REJdu1,0,0,0,0,0,0,0,0,0,0,...,0.764918,1.0,0.090229,0.743976,0.037851,0.212072,0.606,0.418638,0.104717,0.8
0017A9myIXhXYB4gBsPRsA,0,0,0,0,0,0,0,0,0,0,...,0.901301,1.0,0.030042,0.084336,0.000004,0.105732,0.360,0.354490,0.069198,0.8
001reycpvPkUjmpxTJv6eV,0,0,0,0,0,0,0,0,0,0,...,0.662231,0.0,0.036175,0.834337,0.919679,0.117885,0.540,0.488358,0.059419,0.8
002H0J6N0wKjjRtFQhKDzZ,0,0,0,0,0,0,0,0,0,0,...,0.711602,0.0,0.037942,0.530120,0.324297,0.334616,0.752,0.584514,0.097124,0.8


(38456, 108)

Overall there are 108 features used for the Recommendation system.

Try different distance measures / similarity functions.

In [37]:
from sklearn.metrics.pairwise import cosine_similarity, linear_kernel, polynomial_kernel, sigmoid_kernel

Compute the cosine similarity between each song. A value close to one means that 2 two songs are equal a value close to 0 means, that they are completley different.

Cosine similarity

In [38]:
cosine_similarity_array = cosine_similarity(ct_merged)

In [39]:
cosine_similarity_array.shape

(38456, 38456)

In [40]:
cosine_similarity_df = pd.DataFrame(cosine_similarity_array, index=ct_merged.index, columns=ct_merged.index)

In [41]:
cosine_similarity_df

spotify_id,000P83HDtOHcNVFZy7Q2Yu,000paUq6xoYcvCz2REJdu1,0017A9myIXhXYB4gBsPRsA,001reycpvPkUjmpxTJv6eV,002H0J6N0wKjjRtFQhKDzZ,002Ra2MFY7R5ci6q1YLWLH,003787JNriexFRPKRwyMTn,003FCWl5ci9MxCaiTbErSr,004223m8Zb9ZHcxHFxlpQc,006XjVYo0HbGrUfvL4FodP,...,7zn1gpif2DHi9t9isA5Boe,7znOzcWy38z63lIcKOQBd4,7zndxriU3fd1TkV9BMdv5Y,7zraaKkjSygFzRAObStH8C,7zrlCQpCrfAxWEop9J8OzK,7zsH8mPq4nPBqqqDJpjgJe,7zv3sIP0Umtz9MswoJSF1T,7zwHfIlTAIPa6hgjzK1MGx,7zwlOlX3MA7BFkmf0c1y4p,7zzdiBe3usdBtyqHEekCEs
spotify_id,,,,,,,,,,,,,,,,,,,,,
000P83HDtOHcNVFZy7Q2Yu,1.000000,0.584950,0.547657,0.577832,0.615026,0.733066,0.436763,0.696065,0.604715,0.440115,...,0.653633,0.571652,0.590742,0.704053,0.510880,0.656446,0.619765,0.633239,0.511864,0.490575
000paUq6xoYcvCz2REJdu1,0.584950,1.000000,0.850264,0.483625,0.521155,0.757498,0.552569,0.725537,0.476348,0.562444,...,0.712973,0.616512,0.877059,0.708038,0.574449,0.723518,0.538335,0.676929,0.567409,0.531816
0017A9myIXhXYB4gBsPRsA,0.547657,0.850264,1.000000,0.398664,0.490335,0.702231,0.499048,0.689698,0.500751,0.492929,...,0.733256,0.654258,0.722472,0.733983,0.611845,0.790129,0.437428,0.627054,0.495387,0.591632
001reycpvPkUjmpxTJv6eV,0.577832,0.483625,0.398664,1.000000,0.554062,0.586211,0.539577,0.549387,0.429293,0.606362,...,0.529015,0.422936,0.467465,0.509193,0.377041,0.491403,0.569164,0.656092,0.570760,0.369105
002H0J6N0wKjjRtFQhKDzZ,0.615026,0.521155,0.490335,0.554062,1.000000,0.576255,0.427358,0.589067,0.504267,0.378358,...,0.576012,0.461075,0.497093,0.582196,0.451617,0.551722,0.543438,0.599098,0.459383,0.575584
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
7zsH8mPq4nPBqqqDJpjgJe,0.656446,0.723518,0.790129,0.491403,0.551722,0.853721,0.366343,0.582482,0.536233,0.379025,...,0.557212,0.524365,0.682702,0.606600,0.443794,1.000000,0.536788,0.524020,0.413029,0.707870
7zv3sIP0Umtz9MswoJSF1T,0.619765,0.538335,0.437428,0.569164,0.543438,0.659313,0.482791,0.584925,0.439193,0.520979,...,0.518325,0.690803,0.512768,0.522064,0.594918,0.536788,1.000000,0.551991,0.509148,0.411959
7zwHfIlTAIPa6hgjzK1MGx,0.633239,0.676929,0.627054,0.656092,0.599098,0.614871,0.688482,0.765184,0.495488,0.571192,...,0.758185,0.627979,0.635396,0.736928,0.587861,0.524020,0.551991,1.000000,0.723465,0.384070


Linear kernel

In [42]:
linear_kernel_array = linear_kernel(ct_merged)

In [43]:
linear_kernel_array.shape

(38456, 38456)

In [44]:
linear_kernel_df = pd.DataFrame(linear_kernel_array, index=ct_merged.index, columns=ct_merged.index)
linear_kernel_df.head()

spotify_id,000P83HDtOHcNVFZy7Q2Yu,000paUq6xoYcvCz2REJdu1,0017A9myIXhXYB4gBsPRsA,001reycpvPkUjmpxTJv6eV,002H0J6N0wKjjRtFQhKDzZ,002Ra2MFY7R5ci6q1YLWLH,003787JNriexFRPKRwyMTn,003FCWl5ci9MxCaiTbErSr,004223m8Zb9ZHcxHFxlpQc,006XjVYo0HbGrUfvL4FodP,...,7zn1gpif2DHi9t9isA5Boe,7znOzcWy38z63lIcKOQBd4,7zndxriU3fd1TkV9BMdv5Y,7zraaKkjSygFzRAObStH8C,7zrlCQpCrfAxWEop9J8OzK,7zsH8mPq4nPBqqqDJpjgJe,7zv3sIP0Umtz9MswoJSF1T,7zwHfIlTAIPa6hgjzK1MGx,7zwlOlX3MA7BFkmf0c1y4p,7zzdiBe3usdBtyqHEekCEs
spotify_id,,,,,,,,,,,,,,,,,,,,,
000P83HDtOHcNVFZy7Q2Yu,4.256930,2.953650,2.324010,2.831302,2.812805,3.180754,2.155379,3.570523,3.197774,2.039307,...,3.181161,2.756306,3.610660,3.685095,2.730652,2.445887,2.338063,3.375093,2.689898,2.574915
000paUq6xoYcvCz2REJdu1,2.953650,5.989404,4.279828,2.810841,2.827201,3.898633,3.234504,4.414538,2.987895,3.091287,...,4.115933,3.525990,6.358596,4.395858,3.642021,3.197646,2.408937,4.279614,3.536887,3.311023
0017A9myIXhXYB4gBsPRsA,2.324010,4.279828,4.230210,1.947260,2.235485,3.037384,2.455009,3.526743,2.639685,2.276847,...,3.557461,3.144690,4.401928,3.829674,3.260027,2.934732,1.645009,3.331622,2.595126,3.095574
001reycpvPkUjmpxTJv6eV,2.831302,2.810841,1.947260,5.639912,2.916704,2.927719,3.064924,3.243758,2.612997,3.233974,...,2.963520,2.347248,3.288713,3.067704,2.319659,2.107479,2.471463,4.025043,3.452413,2.229948
002H0J6N0wKjjRtFQhKDzZ,2.812805,2.827201,2.235485,2.916704,4.913558,2.686284,2.265792,3.246359,2.864889,1.883516,...,3.011848,2.388455,3.264198,3.273877,2.593389,2.208554,2.202567,3.430563,2.593617,3.245755


In [45]:
df.sort_values('popularity', ascending=False).head(10)

,name,artists,album,release_date,release_date_precision,chart_power,spotify_id,uri,popularity,danceability,...,speechiness,acousticness,instrumentalness,liveness,valence,tempo,duration_ms,time_signature,isrc,genres
239560,Back In Black,AC/DC,Back In Black,1980-07-25,day,NaN,08mG3Y1vljYA6bvDt4Wqkj,spotify:track:08mG3Y1vljYA6bvDt4Wqkj,82,0.310,...,0.0470,0.01100,0.009650,0.0828,0.763,188.386,255493,4,AUAP08000046,rock
126862,Back In Black,AC/DC,Back In Black,1980-07-25,day,NaN,08mG3Y1vljYA6bvDt4Wqkj,spotify:track:08mG3Y1vljYA6bvDt4Wqkj,82,0.310,...,0.0470,0.01100,0.009650,0.0828,0.763,188.386,255493,4,AUAP08000046,hard-rock
237518,You Shook Me All Night Long,AC/DC,Back In Black,1980-07-25,day,901.0,2SiXAy7TuUkycRVbbWDEpo,spotify:track:2SiXAy7TuUkycRVbbWDEpo,81,0.532,...,0.0574,0.00287,0.000513,0.3900,0.755,127.361,210173,4,AUAP08000047,rock
126560,You Shook Me All Night Long,AC/DC,Back In Black,1980-07-25,day,901.0,2SiXAy7TuUkycRVbbWDEpo,spotify:track:2SiXAy7TuUkycRVbbWDEpo,80,0.532,...,0.0574,0.00287,0.000513,0.3900,0.755,127.361,210173,4,AUAP08000047,hard-rock
236291,Could You Be Loved,Bob Marley & The Wailers,Uprising,1980,year,1823.0,5O4erNlJ74PIF6kGol1ZrC,spotify:track:5O4erNlJ74PIF6kGol1ZrC,79,0.916,...,0.1000,0.36000,0.000160,0.0958,0.760,103.312,237000,4,USIR28000016,reggae
196916,9 to 5,Dolly Parton,9 To 5 And Odd Jobs,1980,year,548.0,4w3tQBXhn5345eUXDGBWZG,spotify:track:4w3tQBXhn5345eUXDGBWZG,77,0.554,...,0.0457,0.41600,0.000002,0.6310,0.813,105.390,162467,4,USRN19400384,pop
65758,9 to 5,Dolly Parton,9 To 5 And Odd Jobs,1980,year,548.0,4w3tQBXhn5345eUXDGBWZG,spotify:track:4w3tQBXhn5345eUXDGBWZG,76,0.554,...,0.0457,0.41600,0.000002,0.6310,0.813,105.390,162467,4,USRN19400384,country
126562,Ace of Spades,Motörhead,Ace of Spades (Expanded Edition),1980-11-08,day,NaN,6EPRKhUOdiFSQwGBRBbvsZ,spotify:track:6EPRKhUOdiFSQwGBRBbvsZ,74,0.342,...,0.0758,0.00004,0.000121,0.0823,0.547,140.452,166360,4,GBAJE8000033,hard-rock
151771,Ace of Spades,Motörhead,Ace of Spades (Expanded Edition),1980-11-08,day,NaN,6EPRKhUOdiFSQwGBRBbvsZ,spotify:track:6EPRKhUOdiFSQwGBRBbvsZ,74,0.342,...,0.0758,0.00004,0.000121,0.0823,0.547,140.452,166360,4,GBAJE8000033,metal
287176,The Winner Takes It All,ABBA,Super Trouper,1980,year,1761.0,3oEkrIfXfSh9zGnE7eBzSV,spotify:track:3oEkrIfXfSh9zGnE7eBzSV,74,0.454,...,0.0371,0.57400,0.000766,0.0859,0.520,126.150,294720,4,SEAYD8001020,swedish


In [46]:
def recommend_tracks(track: str, df_rec: pd.DataFrame):
    '''
    Recommends tracks that are similar to the provided track.

    Parameter
    ---------
    track: str
        Provided track

    df: pd.DataFrame
        DataFrame used for the Recommendation
    
    '''
    ids = df[df.name == track]['spotify_id'].values
    for id in ids:
        recommender_df = df_rec[[ids[0]]]
        recommender_df.reset_index(inplace=True)
        recommender_df = recommender_df.rename(columns={id: 'Score'})
        recommender_df = recommender_df.merge(df_total, how='left', on='spotify_id')
        display(recommender_df.sort_values(by='Score', ascending=False).head(10))


In [47]:
recommend_tracks('Back In Black', cosine_similarity_df)

,spotify_id,Score,genres,name,artists,album,release_date,release_date_precision,uri,isrc,...,loudness,mode,speechiness,acousticness,instrumentalness,liveness,valence,tempo,duration_ms,time_signature
722,08mG3Y1vljYA6bvDt4Wqkj,1.000000,"[hard-rock, rock]","[Back In Black, Back In Black]","[AC/DC, AC/DC]","[Back In Black, Back In Black]","[1980-07-25, 1980-07-25]","[day, day]","[spotify:track:08mG3Y1vljYA6bvDt4Wqkj, spotify...","[AUAP08000046, AUAP08000046]",...,-5.678,1.0,0.0470,0.011000,0.009650,0.0828,0.763,188.386,255493.0,4.0
25422,5AD7oNm0kvWIXxW3vy0z2a,0.996716,"[hard-rock, rock]","[Homemade Love, Homemade Love]","[Journey, Journey]","[Departure, Departure]","[1980, 1980]","[year, year]","[spotify:track:5AD7oNm0kvWIXxW3vy0z2a, spotify...","[USSM18000101, USSM18000101]",...,-7.733,1.0,0.0935,0.024400,0.000009,0.1690,0.758,178.672,172560.0,4.0
26519,5O4rVUjsG66l2nArhYW57m,0.996298,"[hard-rock, rock]","[Relentless, Relentless]","[Kansas, Kansas]","[Audio-Visions, Audio-Visions]","[1980-09-01, 1980-09-01]","[day, day]","[spotify:track:5O4rVUjsG66l2nArhYW57m, spotify...","[USSM11102555, USSM11102555]",...,-6.793,1.0,0.0338,0.000872,0.000003,0.0543,0.749,139.839,296813.0,4.0
6199,1En7X8Ama1U8MosJbDndnB,0.992350,"[hard-rock, rock]","[Too Far Gone, Too Far Gone]","[The Babys, The Babys]","[On the Edge, On the Edge]","[1980-10-01, 1980-10-01]","[day, day]","[spotify:track:1En7X8Ama1U8MosJbDndnB, spotify...","[GBAYK0700073, GBAYK0700073]",...,-6.881,1.0,0.0602,0.019700,0.093000,0.1910,0.930,169.456,175000.0,4.0
18529,3m3hmciMkd4y3XGC1mjTHf,0.991354,"[hard-rock, rock]","[Rolê da Estrada, Rolê da Estrada]","[Patrulha do Espaço, Patrulha do Espaço]","[Patrulha do Espaço, Patrulha do Espaço]","[1980-10-10, 1980-10-10]","[day, day]","[spotify:track:3m3hmciMkd4y3XGC1mjTHf, spotify...","[BR3QI1300004, BR3QI1300004]",...,-4.075,1.0,0.0358,0.032100,0.010300,0.1040,0.742,148.813,142000.0,4.0
36341,7MxmJmW08fXXIx0WV1UeKz,0.990243,"[hard-rock, rock]","[Someone Tonight, Someone Tonight]","[REO Speedwagon, REO Speedwagon]","[Hi Infidelity (30th Anniversary Edition), Hi ...","[1980, 1980]","[year, year]","[spotify:track:7MxmJmW08fXXIx0WV1UeKz, spotify...","[USSM11102624, USSM11102624]",...,-5.425,1.0,0.0437,0.075100,0.000000,0.1330,0.855,133.617,160240.0,4.0
25670,5DVXOyNMl4MunWZ2BDvgA5,0.989644,"[hard-rock, rock]","[Hello America, Hello America]","[Def Leppard, Def Leppard]","[On Through The Night (Remastered), On Through...","[1980-03-14, 1980-03-14]","[day, day]","[spotify:track:5DVXOyNMl4MunWZ2BDvgA5, spotify...","[GBUM71802087, GBUM71802087]",...,-6.776,1.0,0.0827,0.001990,0.195000,0.1140,0.635,155.783,207520.0,4.0
25215,57hFsLrkALFFIxncLAY7Tg,0.989644,"[hard-rock, rock]","[Hello America, Hello America]","[Def Leppard, Def Leppard]","[On Through The Night, On Through The Night]","[1980-03-14, 1980-03-14]","[day, day]","[spotify:track:57hFsLrkALFFIxncLAY7Tg, spotify...","[GBF087900785, GBF087900785]",...,-6.776,1.0,0.0827,0.001990,0.195000,0.1140,0.635,155.783,207520.0,4.0
16911,3R306PKKOPAUjSSMkDHOpY,0.988208,"[hard-rock, rock]","[Rock n' Roll Is Alive and Well, Rock n' Roll ...","[The Babys, The Babys]","[On the Edge, On the Edge]","[1980-10-01, 1980-10-01]","[day, day]","[spotify:track:3R306PKKOPAUjSSMkDHOpY, spotify...","[GBAYK8000176, GBAYK8000176]",...,-6.773,1.0,0.0458,0.026500,0.000008,0.2780,0.539,172.996,251600.0,4.0
21489,4NyZk1OfjwkGiIQxsD10ii,0.987438,"[hard-rock, rock]","[Anytime, Anytime]","[The Babys, The Babys]","[Union Jacks, Union Jacks]","[1980-01-01, 1980-01-01]","[day, day]","[spotify:track:4NyZk1OfjwkGiIQxsD10ii, spotify...","[GBAYK0700064, GBAYK0700064]",...,-5.763,1.0,0.0304,0.008980,0.000000,0.1760,0.767,112.699,202760.0,4.0


,spotify_id,Score,genres,name,artists,album,release_date,release_date_precision,uri,isrc,...,loudness,mode,speechiness,acousticness,instrumentalness,liveness,valence,tempo,duration_ms,time_signature
722,08mG3Y1vljYA6bvDt4Wqkj,1.000000,"[hard-rock, rock]","[Back In Black, Back In Black]","[AC/DC, AC/DC]","[Back In Black, Back In Black]","[1980-07-25, 1980-07-25]","[day, day]","[spotify:track:08mG3Y1vljYA6bvDt4Wqkj, spotify...","[AUAP08000046, AUAP08000046]",...,-5.678,1.0,0.0470,0.011000,0.009650,0.0828,0.763,188.386,255493.0,4.0
25422,5AD7oNm0kvWIXxW3vy0z2a,0.996716,"[hard-rock, rock]","[Homemade Love, Homemade Love]","[Journey, Journey]","[Departure, Departure]","[1980, 1980]","[year, year]","[spotify:track:5AD7oNm0kvWIXxW3vy0z2a, spotify...","[USSM18000101, USSM18000101]",...,-7.733,1.0,0.0935,0.024400,0.000009,0.1690,0.758,178.672,172560.0,4.0
26519,5O4rVUjsG66l2nArhYW57m,0.996298,"[hard-rock, rock]","[Relentless, Relentless]","[Kansas, Kansas]","[Audio-Visions, Audio-Visions]","[1980-09-01, 1980-09-01]","[day, day]","[spotify:track:5O4rVUjsG66l2nArhYW57m, spotify...","[USSM11102555, USSM11102555]",...,-6.793,1.0,0.0338,0.000872,0.000003,0.0543,0.749,139.839,296813.0,4.0
6199,1En7X8Ama1U8MosJbDndnB,0.992350,"[hard-rock, rock]","[Too Far Gone, Too Far Gone]","[The Babys, The Babys]","[On the Edge, On the Edge]","[1980-10-01, 1980-10-01]","[day, day]","[spotify:track:1En7X8Ama1U8MosJbDndnB, spotify...","[GBAYK0700073, GBAYK0700073]",...,-6.881,1.0,0.0602,0.019700,0.093000,0.1910,0.930,169.456,175000.0,4.0
18529,3m3hmciMkd4y3XGC1mjTHf,0.991354,"[hard-rock, rock]","[Rolê da Estrada, Rolê da Estrada]","[Patrulha do Espaço, Patrulha do Espaço]","[Patrulha do Espaço, Patrulha do Espaço]","[1980-10-10, 1980-10-10]","[day, day]","[spotify:track:3m3hmciMkd4y3XGC1mjTHf, spotify...","[BR3QI1300004, BR3QI1300004]",...,-4.075,1.0,0.0358,0.032100,0.010300,0.1040,0.742,148.813,142000.0,4.0
36341,7MxmJmW08fXXIx0WV1UeKz,0.990243,"[hard-rock, rock]","[Someone Tonight, Someone Tonight]","[REO Speedwagon, REO Speedwagon]","[Hi Infidelity (30th Anniversary Edition), Hi ...","[1980, 1980]","[year, year]","[spotify:track:7MxmJmW08fXXIx0WV1UeKz, spotify...","[USSM11102624, USSM11102624]",...,-5.425,1.0,0.0437,0.075100,0.000000,0.1330,0.855,133.617,160240.0,4.0
25670,5DVXOyNMl4MunWZ2BDvgA5,0.989644,"[hard-rock, rock]","[Hello America, Hello America]","[Def Leppard, Def Leppard]","[On Through The Night (Remastered), On Through...","[1980-03-14, 1980-03-14]","[day, day]","[spotify:track:5DVXOyNMl4MunWZ2BDvgA5, spotify...","[GBUM71802087, GBUM71802087]",...,-6.776,1.0,0.0827,0.001990,0.195000,0.1140,0.635,155.783,207520.0,4.0
25215,57hFsLrkALFFIxncLAY7Tg,0.989644,"[hard-rock, rock]","[Hello America, Hello America]","[Def Leppard, Def Leppard]","[On Through The Night, On Through The Night]","[1980-03-14, 1980-03-14]","[day, day]","[spotify:track:57hFsLrkALFFIxncLAY7Tg, spotify...","[GBF087900785, GBF087900785]",...,-6.776,1.0,0.0827,0.001990,0.195000,0.1140,0.635,155.783,207520.0,4.0
16911,3R306PKKOPAUjSSMkDHOpY,0.988208,"[hard-rock, rock]","[Rock n' Roll Is Alive and Well, Rock n' Roll ...","[The Babys, The Babys]","[On the Edge, On the Edge]","[1980-10-01, 1980-10-01]","[day, day]","[spotify:track:3R306PKKOPAUjSSMkDHOpY, spotify...","[GBAYK8000176, GBAYK8000176]",...,-6.773,1.0,0.0458,0.026500,0.000008,0.2780,0.539,172.996,251600.0,4.0
21489,4NyZk1OfjwkGiIQxsD10ii,0.987438,"[hard-rock, rock]","[Anytime, Anytime]","[The Babys, The Babys]","[Union Jacks, Union Jacks]","[1980-01-01, 1980-01-01]","[day, day]","[spotify:track:4NyZk1OfjwkGiIQxsD10ii, spotify...","[GBAYK0700064, GBAYK0700064]",...,-5.763,1.0,0.0304,0.008980,0.000000,0.1760,0.767,112.699,202760.0,4.0


In [48]:
recommend_tracks('Back In Black', linear_kernel_df)

,spotify_id,Score,genres,name,artists,album,release_date,release_date_precision,uri,isrc,...,loudness,mode,speechiness,acousticness,instrumentalness,liveness,valence,tempo,duration_ms,time_signature
2112,0PLzoe4izqluPTVsF9K4mb,7.155934,"[hard-rock, hardcore, punk, punk-rock, rock]","[Ennen kolmatta maailmansotaa, Ennen kolmatta ...","[Maukka Perusjätkä,Ralf Örn Pop, Maukka Perusj...","[Ennen kolmatta maailmansotaa, Ennen kolmatta ...","[1980-12-12, 1980-12-12, 1980-12-12, 1980-12-1...","[day, day, day, day, day]","[spotify:track:0PLzoe4izqluPTVsF9K4mb, spotify...","[FILKA8000375, FILKA8000375, FILKA8000375, FIL...",...,-4.752,1.0,0.0726,0.005900,0.000000,0.3390,0.745,173.004,165680.0,4.0
4650,0vEEhp1BFWt1iCKsojoebd,7.150820,"[hard-rock, hardcore, punk, punk-rock, rock]","[Ska 4, Ska 4, Ska 4, Ska 4, Ska 4]","[Maukka Perusjätkä,Ralf Örn Pop, Maukka Perusj...","[Ennen kolmatta maailmansotaa, Ennen kolmatta ...","[1980-12-12, 1980-12-12, 1980-12-12, 1980-12-1...","[day, day, day, day, day]","[spotify:track:0vEEhp1BFWt1iCKsojoebd, spotify...","[FILKA8000369, FILKA8000369, FILKA8000369, FIL...",...,-4.207,1.0,0.2660,0.005240,0.016700,0.1700,0.812,170.904,141800.0,4.0
6199,1En7X8Ama1U8MosJbDndnB,7.114456,"[hard-rock, rock]","[Too Far Gone, Too Far Gone]","[The Babys, The Babys]","[On the Edge, On the Edge]","[1980-10-01, 1980-10-01]","[day, day]","[spotify:track:1En7X8Ama1U8MosJbDndnB, spotify...","[GBAYK0700073, GBAYK0700073]",...,-6.881,1.0,0.0602,0.019700,0.093000,0.1910,0.930,169.456,175000.0,4.0
1578,0Ix7jl28ovNUGclUmkwHnx,7.092672,"[hard-rock, rock]","[My Life, My Life]","[Peter Criss, Peter Criss]","[Out Of Control, Out Of Control]","[1980, 1980]","[year, year]","[spotify:track:0Ix7jl28ovNUGclUmkwHnx, spotify...","[USMR18070057, USMR18070057]",...,-5.082,1.0,0.0339,0.147000,0.000007,0.0634,0.877,119.227,222933.0,4.0
28209,5joQhLd0l0jc1S0U9TkmsG,7.073923,"[alt-rock, alternative, blues, hard-rock, hard...","[Soul Kitchen, Soul Kitchen, Soul Kitchen, Sou...","[X, X, X, X, X, X, X, X, X]","[Los Angeles, Los Angeles, Los Angeles, Los An...","[1980-04-26, 1980-04-26, 1980-04-26, 1980-04-2...","[day, day, day, day, day, day, day, day, day]","[spotify:track:5joQhLd0l0jc1S0U9TkmsG, spotify...","[USFP71869503, USFP71869503, USFP71869503, USF...",...,-4.304,1.0,0.0442,0.002030,0.001110,0.3630,0.836,101.277,145467.0,4.0
29747,63YhDDyYQJo847NqN7mry3,7.066416,"[alt-rock, alternative, blues, grunge, hard-ro...","[Cover Band - Demo, Cover Band - Demo, Cover B...","[Redd Kross, Redd Kross, Redd Kross, Redd Kros...","[Red Cross, Red Cross, Red Cross, Red Cross, R...","[1980-01-01, 1980-01-01, 1980-01-01, 1980-01-0...","[day, day, day, day, day, day, day, day, day, ...","[spotify:track:63YhDDyYQJo847NqN7mry3, spotify...","[USMRG2073408, USMRG2073408, USMRG2073408, USM...",...,-5.967,1.0,0.0743,0.000029,0.002410,0.1480,0.719,182.770,89960.0,4.0
17610,3Zw7u9z5sfbasLy9zkubzn,7.045456,"[blues, british, hard-rock, metal, rock]","[Carry Your Load, Carry Your Load, Carry Your ...","[Whitesnake, Whitesnake, Whitesnake, Whitesnak...","[Ready An' Willing, Ready An' Willing, Ready A...","[1980-01-01, 1980-01-01, 1980-01-01, 1980-01-0...","[day, day, day, day, day]","[spotify:track:3Zw7u9z5sfbasLy9zkubzn, spotify...","[CAU111120901, CAU111120901, CAU111120901, CAU...",...,-4.309,1.0,0.0312,0.048200,0.000004,0.0333,0.897,150.405,244520.0,4.0
30396,6BUuoR1kG1rRag718ibuOn,7.043061,"[country, folk, hard-rock, rock]","[Betty Lou's Gettin' Out Tonight, Betty Lou's ...","[Bob Seger, Bob Seger, Bob Seger, Bob Seger]","[Against The Wind, Against The Wind, Against T...","[1980-02-25, 1980-02-25, 1980-02-25, 1980-02-25]","[day, day, day, day]","[spotify:track:6BUuoR1kG1rRag718ibuOn, spotify...","[USCA28500277, USCA28500277, USCA28500277, USC...",...,-3.081,1.0,0.0506,0.142000,0.126000,0.2930,0.972,170.199,171920.0,4.0
12883,2cXADTZcYNhvtfZuK77Xt0,7.040993,"[blues, hard-rock, metal, rock]","[Hungry Boys, Hungry Boys, Hungry Boys, Hungry...","[Blue Öyster Cult, Blue Öy

,spotify_id,Score,genres,name,artists,album,release_date,release_date_precision,uri,isrc,...,loudness,mode,speechiness,acousticness,instrumentalness,liveness,valence,tempo,duration_ms,time_signature
2112,0PLzoe4izqluPTVsF9K4mb,7.155934,"[hard-rock, hardcore, punk, punk-rock, rock]","[Ennen kolmatta maailmansotaa, Ennen kolmatta ...","[Maukka Perusjätkä,Ralf Örn Pop, Maukka Perusj...","[Ennen kolmatta maailmansotaa, Ennen kolmatta ...","[1980-12-12, 1980-12-12, 1980-12-12, 1980-12-1...","[day, day, day, day, day]","[spotify:track:0PLzoe4izqluPTVsF9K4mb, spotify...","[FILKA8000375, FILKA8000375, FILKA8000375, FIL...",...,-4.752,1.0,0.0726,0.005900,0.000000,0.3390,0.745,173.004,165680.0,4.0
4650,0vEEhp1BFWt1iCKsojoebd,7.150820,"[hard-rock, hardcore, punk, punk-rock, rock]","[Ska 4, Ska 4, Ska 4, Ska 4, Ska 4]","[Maukka Perusjätkä,Ralf Örn Pop, Maukka Perusj...","[Ennen kolmatta maailmansotaa, Ennen kolmatta ...","[1980-12-12, 1980-12-12, 1980-12-12, 1980-12-1...","[day, day, day, day, day]","[spotify:track:0vEEhp1BFWt1iCKsojoebd, spotify...","[FILKA8000369, FILKA8000369, FILKA8000369, FIL...",...,-4.207,1.0,0.2660,0.005240,0.016700,0.1700,0.812,170.904,141800.0,4.0
6199,1En7X8Ama1U8MosJbDndnB,7.114456,"[hard-rock, rock]","[Too Far Gone, Too Far Gone]","[The Babys, The Babys]","[On the Edge, On the Edge]","[1980-10-01, 1980-10-01]","[day, day]","[spotify:track:1En7X8Ama1U8MosJbDndnB, spotify...","[GBAYK0700073, GBAYK0700073]",...,-6.881,1.0,0.0602,0.019700,0.093000,0.1910,0.930,169.456,175000.0,4.0
1578,0Ix7jl28ovNUGclUmkwHnx,7.092672,"[hard-rock, rock]","[My Life, My Life]","[Peter Criss, Peter Criss]","[Out Of Control, Out Of Control]","[1980, 1980]","[year, year]","[spotify:track:0Ix7jl28ovNUGclUmkwHnx, spotify...","[USMR18070057, USMR18070057]",...,-5.082,1.0,0.0339,0.147000,0.000007,0.0634,0.877,119.227,222933.0,4.0
28209,5joQhLd0l0jc1S0U9TkmsG,7.073923,"[alt-rock, alternative, blues, hard-rock, hard...","[Soul Kitchen, Soul Kitchen, Soul Kitchen, Sou...","[X, X, X, X, X, X, X, X, X]","[Los Angeles, Los Angeles, Los Angeles, Los An...","[1980-04-26, 1980-04-26, 1980-04-26, 1980-04-2...","[day, day, day, day, day, day, day, day, day]","[spotify:track:5joQhLd0l0jc1S0U9TkmsG, spotify...","[USFP71869503, USFP71869503, USFP71869503, USF...",...,-4.304,1.0,0.0442,0.002030,0.001110,0.3630,0.836,101.277,145467.0,4.0
29747,63YhDDyYQJo847NqN7mry3,7.066416,"[alt-rock, alternative, blues, grunge, hard-ro...","[Cover Band - Demo, Cover Band - Demo, Cover B...","[Redd Kross, Redd Kross, Redd Kross, Redd Kros...","[Red Cross, Red Cross, Red Cross, Red Cross, R...","[1980-01-01, 1980-01-01, 1980-01-01, 1980-01-0...","[day, day, day, day, day, day, day, day, day, ...","[spotify:track:63YhDDyYQJo847NqN7mry3, spotify...","[USMRG2073408, USMRG2073408, USMRG2073408, USM...",...,-5.967,1.0,0.0743,0.000029,0.002410,0.1480,0.719,182.770,89960.0,4.0
17610,3Zw7u9z5sfbasLy9zkubzn,7.045456,"[blues, british, hard-rock, metal, rock]","[Carry Your Load, Carry Your Load, Carry Your ...","[Whitesnake, Whitesnake, Whitesnake, Whitesnak...","[Ready An' Willing, Ready An' Willing, Ready A...","[1980-01-01, 1980-01-01, 1980-01-01, 1980-01-0...","[day, day, day, day, day]","[spotify:track:3Zw7u9z5sfbasLy9zkubzn, spotify...","[CAU111120901, CAU111120901, CAU111120901, CAU...",...,-4.309,1.0,0.0312,0.048200,0.000004,0.0333,0.897,150.405,244520.0,4.0
30396,6BUuoR1kG1rRag718ibuOn,7.043061,"[country, folk, hard-rock, rock]","[Betty Lou's Gettin' Out Tonight, Betty Lou's ...","[Bob Seger, Bob Seger, Bob Seger, Bob Seger]","[Against The Wind, Against The Wind, Against T...","[1980-02-25, 1980-02-25, 1980-02-25, 1980-02-25]","[day, day, day, day]","[spotify:track:6BUuoR1kG1rRag718ibuOn, spotify...","[USCA28500277, USCA28500277, USCA28500277, USC...",...,-3.081,1.0,0.0506,0.142000,0.126000,0.2930,0.972,170.199,171920.0,4.0
12883,2cXADTZcYNhvtfZuK77Xt0,7.040993,"[blues, hard-rock, metal, rock]","[Hungry Boys, Hungry Boys, Hungry Boys, Hungry...","[Blue Öyster Cult, Blue Öy